# **ChatBot desenvolvido para a matéria de IA,**

## **Escopo:**
> ChatBot em desenvolvimento voltado para vendas de produtos de uma papelaria.






*   Aluno: Iago Akio Saito
*   RGA: 2020.1906.060-1



In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import make_scorer
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
import nltk
import nltk
import pandas as pd
import numpy as np
import unicodedata


url = "https://raw.githubusercontent.com/Iagoakiosaito/IA-ChatBot-UFMS/main/Chatter-DB.csv"
df=pd.read_csv(url)
df = df[df['Orador'].notna()]
df = df[df['Sentença'].notna()]
df = df[df['Intenção'].notna()]
df = df[(df.Orador == '<')]

ModuleNotFoundError: No module named 'nltk'

In [14]:
def normalize_caseless(text):
    return unicodedata.normalize("NFKD", text.casefold()).encode("ASCII", "ignore").decode("ASCII")

def caseless_equal(left, right):
    return normalize_caseless(left) == normalize_caseless(right)

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.6, strip_accents='unicode')

In [ ]:
x = vectorizer.fit_transform(df.Sentença)
y = np.array(df.Intenção)

# KNN, sem utilização do loo

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split( x, y, test_size=0.3, random_state=10,stratify=y)
model = KNeighborsClassifier(n_neighbors=8, weights= "uniform")
model.fit(x_treino,y_treino)

text = input(str())
inst = vectorizer.transform([text])
print(model.predict(inst))

#### Métricas KNN

In [ ]:
pred_KNN = model.predict(x_teste)

In [ ]:
metrics.precision_score(y_teste,pred_KNN,average='macro')

1.0

In [ ]:
metrics.recall_score(y_teste,pred_KNN,average='macro')

1.0

# Random Search

In [ ]:
parameters = {'n_neighbors':range(1,10),'weights':['uniform','distance']}

In [ ]:
rs = RandomizedSearchCV(model,parameters,n_iter=10,refit=True)

In [ ]:
rs.fit(x,y)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=8,
                                                  p=2, weights='uniform'),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'n_neighbors': range(1, 10),
                                        'weights': ['uniform', 'distance']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [ ]:
rs.best_params_

{'n_neighbors': 9, 'weights': 'uniform'}

In [ ]:
rs.best_score_

0.95

In [ ]:
rs.best_estimator_

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')

# Árvore de decisão simples, sem utilização do loo

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split( x, y, test_size=0.3, random_state=10,stratify=y)

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_treino, y_treino)
pred_clf = clf.predict(x_teste)

In [ ]:
text = input(str())
inst = vectorizer.transform([text])
print(clf.predict(inst))

lápis
['Pedido']


In [ ]:
metrics.precision_score(y_teste,pred_clf,average="micro")

1.0

In [ ]:
metrics.recall_score(y_teste,pred_clf,average='macro')

1.0

# Leave One Out

## loo do KNN

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(x)
scores = []
for train_index, test_index in loo.split(x):

    x_treino, x_teste = x[train_index], x[test_index]
    y_treino, y_teste = y[train_index], y[test_index]
    
    model = KNeighborsClassifier(n_neighbors=9, weights= "uniform")
    model.fit(x_treino,y_treino)
    predi = model.predict(x_teste)
    scores.append(metrics.accuracy_score(y_teste,predi))
np.mean(scores)

0.9777777777777777

In [ ]:
metrics.precision_score(y_teste, predi, average="macro")

1.0

In [ ]:
metrics.recall_score(y_teste, predi, average='macro')

1.0

## loo da árvore de decisão

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(x)
scores = []
for train_index, test_index in loo.split(x):

    x_treino, x_teste = x[train_index], x[test_index]
    y_treino, y_teste = y[train_index], y[test_index]
    
    clf = tree.DecisionTreeClassifier()
    clf.fit(x_treino,y_treino)
    pred = clf.predict(x_teste)
    scores.append(metrics.accuracy_score(y_teste,pred))
np.mean(scores)

0.9833333333333333

# **Fase 2**

In [9]:

setup_module()


ModuleNotFoundError: No module named 'nltk'

## Etiquetador de entidades

In [15]:
corpus = input(str("Insira aqui o corpus para ter suas entidades identificadas: "))
corpus = normalize_caseless(corpus)

In [1]:
dict_ent = {

    "produto" : ["caderno", "cadernos", "lapis", "livro", "livros", "caneta", "canetas", "canetinha", "canetinhas", "papel", "papeis", "eva", "borracha", "borrachas", ""],
    "quantidade" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    "quantidade" : ["um", "dois", "tres", "quatro", "cinco", "seis", "sete", "oito", "nove", "dez", "onze", "doze", "treze", "quatorze", "quinze", "dezesseis", "dezessete", "dezoito", "dezenove", "vinte"]

}

In [17]:
split = corpus.split()

for palavra in split:
    for chave, valor in dict_ent.items():
        
        

SyntaxError: invalid syntax (Temp/ipykernel_15144/2442418435.py, line 5)